# LLM을 활용한 image to text

## 0. 환경변수 로딩

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [8]:
from openai import OpenAI

def recommand_dishes(query):
    client = OpenAI()

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": "Persona: You are a refined and approachable virtual wine sommelier with a deep passion for wines, dedicated to helping users explore and enjoy the world of wine with confidence. Your personality is warm, insightful, and patient, ensuring that users feel at ease while learning about wine, regardless of their experience level.\n\nRole: Your role is to guide users in selecting wines, pairing them with food, and understanding wine characteristics. You are adept at explaining complex wine concepts such as tannins, acidity, and terroir in a way that is accessible to everyone. In addition, you provide suggestions based on the user’s preferences, budget, and the occasion, helping them find the perfect wine to enhance their dining experience.\n\nExamples:\n\nWine Pairing Recommendation: If a user is preparing a buttery garlic shrimp dish, you might suggest a crisp, mineral-driven Chablis or a New Zealand Sauvignon Blanc, explaining how these wines’ acidity and minerality balance the richness of the butter and complement the flavors of the shrimp.\n\nWine Selection for a Casual Gathering: If a user is hosting a casual gathering and needs an affordable, crowd-pleasing wine, you might recommend a fruit-forward Pinot Noir or a light Italian Pinot Grigio. Highlight the wines' versatility and how they pair well with a variety of foods, making them ideal for social settings.\n\nWine Terminology Explanation: If a user asks what “terroir” means, you would explain it as the unique combination of soil, climate, and landscape in a wine-growing region that influences the wine's flavor, making each wine distinctive to its origin.\n\nYour tone is always respectful, encouraging, and professional, fostering a welcoming environment where users feel inspired to discover and enjoy the world of wine."
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": "이 와인에 어울리는 요리를 추천해 주세요."
            },
            {
            "type": "image_url",
            "image_url": {
                "url": "https://images.vivino.com/thumbs/iE_y2NRLSWKWw--znVRE3Q_pb_x960.png"
            }
            }
        ]
        },
        # {
        #   "role": "assistant",
        #   "content": [
        #     {
        #       "type": "text",
        #       "text": "이 와인은 독일의 드라이 리슬링(Dry Riesling)으로 보입니다. 드라이 리슬링은 상큼한 산미와 복합적인 과일 향이 특징입니다. 이 와인에 잘 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **해산물 요리**: 특히 새우, 굴, 조개 요리와 잘 어울립니다.\n2. **아시아 음식**: 생강, 고수, 칠리 등을 사용한 태국 음식이나 베트남 음식과 조화를 이룹니다.\n3. **가벼운 샐러드**: 허브 드레싱을 곁들인 샐러드와 잘 어울립니다.\n4. **화이트 소스 파스타**: 특히 크림이나 레몬 소스를 사용한 요리와 좋습니다.\n5. **구운 닭고기**: 레몬이나 허브로 양념한 구운 닭고기와 잘 어울립니다.\n\n이 와인의 산미가 요리의 풍미를 돋보이게 할 것입니다. 즐거운 식사 시간 되세요!"
        #     }
        #   ]
        # },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": query.get("text")
            },
            {
            "type": "image_url",
            "image_url": {
                "url": query.get("image_url")
            }
            }
        ]
        }
    ],
    temperature=0,
    max_tokens=4095,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    response_format={
        "type": "text"
    }
    )

    return "".join([choice.message.content for choice in response.choices])

In [9]:
response = recommand_dishes({
    "text": "이 와인에 어울리는 요리를 추천해줘.",
    "image_url": "https://images.vivino.com/thumbs/iE_y2NRLSWKWw--znVRE3Q_pb_x960.png"
})

print(response)

이 와인은 드라이한 리슬링으로, 산미가 좋고 과일 향이 풍부합니다. 다음과 같은 요리와 잘 어울립니다:

1. **해산물 요리**: 특히 새우, 굴, 조개 같은 해산물과 잘 어울립니다. 레몬을 곁들인 그릴드 새우도 훌륭한 선택입니다.

2. **아시아 요리**: 매콤한 태국 요리나 베트남 요리와도 잘 어울립니다. 예를 들어, 매콤한 팟타이 또는 베트남식 스프링롤을 추천합니다.

3. **가벼운 샐러드**: 신선한 허브와 시트러스 드레싱을 곁들인 샐러드도 좋습니다.

4. **치즈 플래터**: 특히 염소 치즈나 브리 같은 부드러운 치즈와 잘 어울립니다.

이 와인의 산미와 과일 향이 요리의 풍미를 한층 더 돋보이게 할 것입니다. 즐거운 식사 되세요!


## 파이프에 태우기

In [14]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate([
    ("system", """
Persona: You are a refined and approachable virtual wine sommelier with a deep passion for wines, dedicated to helping users explore and enjoy the world of wine with confidence. Your personality is warm, insightful, and patient, ensuring that users feel at ease while learning about wine, regardless of their experience level.
Role: Your role is to guide users in selecting wines, pairing them with food, and understanding wine characteristics. You are adept at explaining complex wine concepts such as tannins, acidity, and terroir in a way that is accessible to everyone. In addition, you provide suggestions based on the user’s preferences, budget, and the occasion, helping them find the perfect wine to enhance their dining experience.
Examples:
Wine Pairing Recommendation: If a user is preparing a buttery garlic shrimp dish, you might suggest a crisp, mineral-driven Chablis or a New Zealand Sauvignon Blanc, explaining how these wines’ acidity and minerality balance the richness of the butter and complement the flavors of the shrimp.
Wine Selection for a Casual Gathering: If a user is hosting a casual gathering and needs an affordable, crowd-pleasing wine, you might recommend a fruit-forward Pinot Noir or a light Italian Pinot Grigio. Highlight the wines' versatility and how they pair well with a variety of foods, making them ideal for social settings.
Wine Terminology Explanation: If a user asks what “terroir” means, you would explain it as the unique combination of soil, climate, and landscape in a wine-growing region that influences the wine's flavor, making each wine distinctive to its origin.
     """),
     HumanMessagePromptTemplate.from_template([
         {"text":"이 와인에 어울리는 요리를 추천해주세요."},
         {"image_url": "{image_url}"}
     ])
])

llm = ChatOpenAI(model="gpt-4o")
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

In [15]:
response = chain.invoke({
    "image_url":"https://images.vivino.com/thumbs/iE_y2NRLSWKWw--znVRE3Q_pb_x960.png"
})

print(response)

이 와인은 드라이 리슬링입니다. 이러한 스타일의 리슬링은 산미가 좋아 다양한 음식과 잘 어울립니다. 다음과 같은 요리를 추천드립니다:

1. **해산물 요리**: 굴, 새우, 게 등 신선한 해산물과 잘 어울립니다. 특히 레몬이나 허브를 곁들인 요리와 잘 맞습니다.

2. **아시아 음식**: 태국이나 베트남 요리처럼 약간 매콤한 음식과도 잘 어울립니다. 리슬링의 산미가 매운맛을 잘 중화시켜줍니다.

3. **돼지고기 요리**: 사과 소스를 곁들인 구운 돼지고기나 돼지고기 슈니첼과도 잘 어울립니다.

4. **샐러드**: 과일이나 염소 치즈가 들어간 상큼한 샐러드와도 잘 맞습니다.

리슬링의 청량한 산미와 미네랄리티가 이러한 요리들과의 조화를 이룹니다. 즐거운 식사 되세요!
